<font size=6><b>Lec09. 교차검증
* https://amueller.github.io/aml/04-model-evaluation/1-data-splitting-strategies.html

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ------------------------------------------------------
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
# ------------------------------------------------------

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score 
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import roc_curve, roc_auc_score
# ------------------------------------------------------


import warnings
warnings.filterwarnings(action="ignore")

In [2]:
#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

#-------------------- 차트 관련 속성 (차트크기 ) -----------
# plt.rcParams['figure.figsize'] = (4, 2)
# plt.rcParams['font.size'] = 7

# Data Load
* df.info()
* df.shape
* df.head()

In [3]:
data = load_iris()
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [4]:
df = pd.DataFrame(data=data["data"], columns=data["feature_names"] )  #index=[]
df.columns = ["sl","sw","pl","pw"]
df['target'] = data["target"]
df.head()

,sl,sw,pl,pw,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sl      150 non-null    float64
 1   sw      150 non-null    float64
 2   pl      150 non-null    float64
 3   pw      150 non-null    float64
 4   target  150 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 5.4 KB


In [6]:
df.shape

(150, 5)

# 교차검증(corss-validation)
* Defaults in scikit-learn
    * 5-fold in 0.22 (used to be 3 fold)
    * For classification cross-validation is stratified
    * train_test_split has stratify option: train_test_split(X, y, stratify=y)
    * No shuffle by default!

* enumerate 연습

In [7]:
for v in [1,2,3]:
    print(v)

1
2
3


In [8]:
for loop_cnt, v in enumerate([1,2,3]):
    print(loop_cnt, v)

0 1
1 2
2 3


In [9]:
for loop_cnt, v in enumerate([ [1,10] ,  [2,20] ,  [3,30]]):
    print(loop_cnt, v)

0 [1, 10]
1 [2, 20]
2 [3, 30]


In [10]:
for loop_cnt, (v1,v2) in enumerate([ [1,10] ,  [2,20] ,  [3,30]]):
    print(loop_cnt, v1, v2)

0 1 10
1 2 20
2 3 30


## KFold()
* 회귀모델

In [11]:
from sklearn.model_selection import train_test_split, KFold

```python
class sklearn.model_selection.KFold(n_splits=5, *, shuffle=False, random_state=None)
```

In [12]:
df['target'].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [13]:
y = df['target']
X = df.drop('target', axis=1)

score_list = []
kfold = KFold(n_splits=3, shuffle=True, random_state=1212)
    
for i, (train_idx, test_idx) in enumerate(kfold.split(X)) :
    # print(i, train_idx, test_idx)
    print(df.loc[test_idx, "target"].values)
    print(df.loc[train_idx, "target"].values)
    
    # print(X.loc[train_idx].values)
    # print(X.loc[test_idx].values)
    # print(y.loc[train_idx].values)
    # print(y.loc[test_idx].values)
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
    X_train, X_test,  y_train, y_test = X.loc[train_idx], X.loc[test_idx], y.loc[train_idx], y.loc[test_idx]
    print(i, X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    
    model = DecisionTreeClassifier(random_state=111, min_samples_split=2)
    model.fit(X_train, y_train)
    pred = model.predict( X_test)  
    f1   = f1_score(y_test, pred, average="macro")
    print(f"f1 : {f1:.4f}")

    score_list.append(f1)
    print("---"*30)

print(f"평균점수 : {np.mean(score_list):.4f}")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
0 (100, 4) (50, 4) (100,) (50,)
f1 : 0.9139
------------------------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
1 (100, 4) (50, 4) (100,) (50,)
f1 : 0.9797
------------------------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 

## StratifiedKFold()
* 분류모델
* 학습 시 target 비율에 맞게 데이터를 각 fold에 넣어준다

```python
class sklearn.model_selection.StratifiedKFold(n_splits=5, *, shuffle=False, random_state=None
```

In [14]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

In [15]:
y = df['target']
X = df.drop('target', axis=1)

score_list = []
# kfold = KFold(n_splits=10, shuffle=True, random_state=1212)    
# for i, (train_idx, test_idx) in enumerate(kfold.split(X)) :

skfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1212)  
#---------------------------------------------------------------
# y를 넣어야 비율에 맞게 넣을 수 있다
#---------------------------------------------------------------
for i, (train_idx, test_idx) in enumerate(skfold.split(X, y)) :  
    # print(i, train_idx, test_idx)
    # print(df.loc[test_idx, "target"].values)
    # print(df.loc[train_idx, "target"].values)
    print(i, y[train_idx].value_counts())
    
    # print(X.loc[train_idx].values)
    # print(X.loc[test_idx].values)
    # print(y.loc[train_idx].values)
    # print(y.loc[test_idx].values)
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
    X_train, X_test,  y_train, y_test = X.loc[train_idx], X.loc[test_idx], y.loc[train_idx], y.loc[test_idx]
    print(i, X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    
    model = DecisionTreeClassifier(random_state=111, min_samples_split=2)
    model.fit(X_train, y_train)
    pred = model.predict( X_test)  
    f1   = f1_score(y_test, pred, average="macro")
    print(f"f1 : {f1:.4f}")

    score_list.append(f1)
    print("---"*30)

print(f"평균점수 : {np.mean(score_list):.4f}")

0 2    34
0    33
1    33
Name: target, dtype: int64
0 (100, 4) (50, 4) (100,) (50,)
f1 : 0.9596
------------------------------------------------------------------------------------------
1 1    34
0    33
2    33
Name: target, dtype: int64
1 (100, 4) (50, 4) (100,) (50,)
f1 : 0.9385
------------------------------------------------------------------------------------------
2 0    34
1    33
2    33
Name: target, dtype: int64
2 (100, 4) (50, 4) (100,) (50,)
f1 : 0.9216
------------------------------------------------------------------------------------------
평균점수 : 0.9399


### train_test_split(stratify=타켓피쳐명)
* stratify 속성으로 fold에 골고루 넣어준다

```python
sklearn.model_selection.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
```

In [16]:
y = df['target']
X = df.drop('target', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y_train.value_counts())

(100, 4) (50, 4) (100,) (50,)
0    35
1    33
2    32
Name: target, dtype: int64


In [17]:
y = df['target']
X = df.drop('target', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.33, stratify=y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y_train.value_counts())

(100, 4) (50, 4) (100,) (50,)
2    34
1    33
0    33
Name: target, dtype: int64


## cross_val_score()

```python
sklearn.model_selection.cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)
```

In [18]:
from sklearn.model_selection import cross_val_score

In [19]:
model = DecisionTreeClassifier(random_state=111, min_samples_split=2)
y = df['target']
X = df.drop('target', axis=1)


# -------------------------------------
# cv : (Stratified)KFold
# -------------------------------------
skfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1212)  # 분류
kfold  = KFold(n_splits=3, shuffle=True, random_state=1212)            # 회귀

score_arr = cross_val_score(model, X, y, scoring="f1_macro", cv=skfold) 
print(score_arr)

# print(f"평균점수 : {np.mean(score_list):.4f}")
print(f"평균점수 : {score_arr.mean():.4f}")


[0.95955882 0.93849021 0.92156863]
평균점수 : 0.9399


<pre>
skold  0.9399
kfold  0.9460

## cross_validate()
* cross_val_score() : 점수를 array로 준다
* cross_validate() : dict를 준다(점수, 실행시간, 기타..)

```python
sklearn.model_selection.cross_validate(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', return_train_score=False, return_estimator=False, return_indices=False, error_score=nan)
```

In [20]:
from sklearn.model_selection import cross_validate


In [21]:
model = DecisionTreeClassifier(random_state=111, min_samples_split=2)
y = df['target']
X = df.drop('target', axis=1)


# -------------------------------------
# cv : (Stratified)KFold
# -------------------------------------
skfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1212)  # 분류
kfold  = KFold(n_splits=3, shuffle=True, random_state=1212)            # 회귀

# score_arr = cross_val_score(model, X, y, scoring="f1_macro", cv=skfold) 
score_dict = cross_validate(model, X, y, scoring="f1_macro", cv=skfold) 
print(score_dict)

print(f"평균점수 : {score_dict['test_score'].mean():.4f}")
# print(f"평균점수 : {np.mean(score_list):.4f}")
# print(f"평균점수 : {score_arr.mean():.4f}")

pd.DataFrame(score_dict).head() 

{'fit_time': array([0.00598431, 0.00498605, 0.00299191]), 'score_time': array([0.00550604, 0.00398803, 0.00498509]), 'test_score': array([0.95955882, 0.93849021, 0.92156863])}
평균점수 : 0.9399


,fit_time,score_time,test_score
0,0.005984,0.005506,0.959559
1,0.004986,0.003988,0.938490
2,0.002992,0.004985,0.921569


# GridSearchCV
* 교차검증(corss-validation) + 튜닝(hyper parameter)

``` python
class sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, refit=True, cv=None, verbose=0, n_jobs=None, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)

class sklearn.tree.DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)

```

In [22]:
from sklearn.model_selection import GridSearchCV

In [23]:
model = DecisionTreeClassifier(random_state=111, min_samples_split=2)
y = df['target']
X = df.drop('target', axis=1)
#max_depth=None, min_samples_split=2, min_samples_leaf=1
myparam = { "max_depth" : [10,20,30] ,
            "min_samples_split" : [1,2,3,4],
            "min_samples_leaf" : [1,2,3]
          }

skfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1212)  # 분류
gcv_model = GridSearchCV(model, param_grid=myparam, scoring="f1_macro", refit=True, cv=skfold)
gcv_model.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1212, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=111),
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [1, 2, 3, 4]},
             scoring='f1_macro')

In [28]:
# gcv_model.cv_results_
print( gcv_model.best_estimator_ )
print( gcv_model.best_score_)
print( gcv_model.best_params_)
# print( gcv_model.best_score_)


DecisionTreeClassifier(max_depth=10, random_state=111)
0.9398725551108917
{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [25]:
# refit=False
# dt = DecisionTreeClassifier(max_depth=10, min_samples_split=3, min_samples_leaf=4)
# df.fit()

# refit=True
# gcv_model.fit()
